In [1]:
import cv2
import numpy as np
import os

# KNN Algorithm

In [2]:
def distance(x1, x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(x, y, query, k=5):
    vals = []
    m = x.shape[0]
    for i in range(m):
        d = distance(query, x[i])
        vals.append((d, y[i]))
    vals = sorted(vals)
    vals = vals[:k]
    vals = np.array(vals)
    unique_vals = np.unique(vals[:,1], return_counts=True)
    index = unique_vals[1].argmax()
    pred = unique_vals[0][index]
    return pred
    

# Detect Face using Haarcascade

In [3]:
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

#Better use gray frame if storing large data
face_data = []
labels = []
dataset_path = "Face Dataset\\"

#Essentially the labels for given file
class_id = 0
#For mapping between id and name
names = {}

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        #Mapping class id to name
        names[class_id] = fx[:-4]
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #Assigning labels
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0)
    
while True:
    #Read the video input from front camera
    ret, frame = cap.read()
    if ret == False:
        continue
    
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    if(len(faces)==0):
        continue
    
    for face in faces:
        x, y, w, h = face
        
        offset = 10
        face_section = frame[y-offset: y+h+offset, x-offset: x+w+offset]
        face_section = cv2.resize(face_section, (100,100))
        
        #Predicting
        k = int(face_labels.shape[0]*0.15)
        out = knn(face_dataset, face_labels, face_section.flatten())
        
        pred_name = names[int(out)]
        cv2.putText(frame, pred_name, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 2)
        
    
    cv2.imshow("Camera", frame)
        
    key_pressed = cv2.waitKey(1) & 0xff
    if key_pressed == ord(chr(13)):
        break;

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 